# Radio Light Curve Module

**Lecturer:** David Kaplan<br>
**Jupyter Notebook Authors:** Dougal Dobie, David Kaplan & Cameron Hummels

This is a Jupyter notebook lesson taken from the GROWTH Summer School 2019.  For other lessons and their accompanying lectures, please see: http://growth.caltech.edu/growth-school-2019.html

## Objective
Produce and analyze radio light curve data, fitting a broken power law to it.

## Key steps
- Plot the radio light curve
- Determine its spectral index
- Scale the data based on its spectral index
- Fit the data with a power law
- Fit the data with a broken power law with a smooth turnover

## Required dependencies

See GROWTH school webpage for detailed instructions on how to install these modules and packages.  Nominally, you should be able to install the python modules with `pip install <module>`.  The external astromatic packages are easiest installed using package managers (e.g., `rpm`, `apt-get`).

### Python modules
* python 3
* astropy
* numpy
* scipy
* matplotlib
* emcee
* corner

### External packages
None

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
from astropy.io import ascii
import corner
import os
from timeit import default_timer as timer
import matplotlib.colors as colors
import matplotlib.cm as cmx
from scipy.optimize import least_squares, curve_fit
from scipy.stats import f

# Load the data
We write a basic function to load in the radio lightcurve.  It's pretty easy using `astropy`.  But you should look at the file `radio_lightcurve.dat` and make sure that the data returned look appropriate.  It's also good to keep track of some basic info like:
1. How many observations are there?
2. What is the first observation date?  What is the last?
3. What is the lowest frequency?  What is the highest frequency?
4. Do all of the observations actually report _detections_, where the source is detected at $>3\sigma$ significance?

In [ ]:
def load_data(filename='radio_lightcurve.dat'):
    data = ascii.read(filename)
    return data
os.chdir('data')
data = load_data()

# Make a plot of the radio lightcurve
Plot the flux density as a function of time.   This works best as a log-log plot. Make sure to modularise your code so that we can re-use parts of it later on. For bonus points use different markers for each telescope, and use a colour scale to denote the observation frequency.

In [ ]:
def plot_data(ax, sm, data, scaled=False, **kwargs):
    telescope_marker_dict = {'VLA':'s', 'ATCA':'o', 'GMRT':'d'}
    
    
    for row in data:
        #Loop over each row of the data, set the marker colour based on frequency and the marker style based on telescope
    return

def cmap_setup(cmap='viridis', min_freq=0, max_freq=17):
    '''
    This function will set up a scalar map for you to colour your markers by frequency
    '''
    freq_cmap = plt.cm.get_cmap(cmap)
    
    cNorm  = colors.Normalize(vmin=min_freq, vmax=max_freq)
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
    sm = scalarMap
    sm._A = []
    
    return sm    
    
def make_plot(data, scaled=False, model=None, params=None, tvals=np.arange(10,400), plot_models=False):
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111)
    
    #Get the scalar map, plot the data using the plot_data function above
    sm = cmap_setup()
    plot_data(ax, sm, data, scaled=scaled)
    
    
    #Set up a colourbar

    
    #Set axis scales to log
    
    
    #Label axes, set axis limits etc.
    
make_plot(data)

# Determining the spectral index
As with many things in astrophysics, the emission in the radio regime is _non-thermal_ in origin (unlike the early emission in the optical/UV/infrared, which is a thermal blackbody).  What that means is that generally has a power-law spectrum:
$$ S_\nu(\nu) \propto \nu^\alpha$$
Strictly this only works for data observed at _exactly_ the same time, but real observations don't work that way.  A single telescope can (usually) only observe at a single frequency, and different telescopes are separated in time by schedules, longitude, etc.  So we need to be a bit more generous about how we define simultaneous. Luckily, the light-curve is mostly evolving pretty slowly, so this isn't necessarily a problem.

Write a function to take a subset of the data that was observed roughly simultaneously and calculate the spectral index $\alpha$ and its uncertainty. Using multi-band observation at 162 days post-merger calculate the spectral index.  Questions:
1. How many observations did you use, over what time window?
2. What do you get for $\alpha$ and its uncertainty?
3. Do you think using a finite time window introduced significant errors into this calculation?

In [ ]:
def calc_power_law(freq,S0,alpha):
    S = S0 * (freq) ** alpha
    return S

def alpha_calc(data):
    '''
    Write a function to take in some subset of data and fit a power law to the spectrum using the curve_fit function.
    Your function should return a tuple containing the spectral index and associated uncertainty.
    
    Hint: Don't forget to include uncertainties (and set absolute_sigma=True)
    '''
    
    
    return alpha, alpha_err

In [ ]:
#Select the data at the ~162 day epoch and print the spectral index + uncertainty

sel_data = data[data['delta_t'] == 162.89]

alpha, alpha_err = alpha_calc(sel_data)

print("alpha = %.1f+/-%.1f"%(alpha, alpha_err))

## Scaling the data based on the spectral Index
Based on the $\alpha$ you calculated above, what happens if you assume that _all_ of the observations can be fit using the same frequency power-law?  i.e., if $\alpha$ were the same at all times?  If this is the case then scaling all of the data to a single frequency makes it easier to understand the temporal properties of the lightcurve as a function of only 1 variable, not 2.

Write a function to take the observed data and scale it to a specific frequency based on an estimated spectral index. 

Don't forget to include uncertainties! You should add two columns to your data table called "scaled_flux" and "scaled_rms".

Questions:
1. Can you think of reasons why the spectral index should stay the same?  Why it should change?

In [ ]:
def scale_data(data, alpha, alpha_err, ref_freq=3.0):
    #calculate a scaling factor for the flux density and uncertainty
    f_scale = 
    rms_scale = 
    
    #scale the flux and uncertainty - don't forget to add errors in quadrature
    scaled_flux = 
    scaled_rms = 
    
    #Add two new columns to the data
    data['scaled_flux'] = scaled_flux
    data['scaled_rms'] = scaled_rms
    
    return data

Modify your plot_data() and make_plot() functions to add a keyword parameter "scaled" that is by default False. If scaled=True, plot_data() should plot the scaled data instead of the observed data.

Scale the data to 3 GHz based on your estimated spectral index and associated uncertainty, then plot the scaled lightcurve

In [ ]:
data = scale_data(data, -0.6,0.1)
make_plot(data, scaled=True)

# Fitting (some of) the data
We now want to characterise the radio lightcurve. You should be able to see that it initially rises according to a power law, peaks somewhere between 100 and 200 days post-merger and then declines according to a different power law.

However, when we published the first paper demonstrating evidence of a turnover we only had observations up to 200 days post-merger. We will now determine evidence for a turnover using that subset of data.

Create a new table called tdata with the data up to 200 days post-merger and plot the data using your make_plot() function.

In [ ]:
data = load_data()
data = scale_data(data, -0.6,0.05)
tdata = data[data['delta_t'] < 200]

make_plot(tdata, scaled=True)

We can fit this data with a "smoothed broken power law", which combines two power laws with a smoothing parameter around the break point. One functional form of this is given by

$S_\nu(t) = S_{\rm \nu,peak} \left[ \left(\dfrac{t}{t_{\rm peak}}\right)^{-s\delta_1} + \left(\dfrac{t}{t_{\rm peak}}\right)^{-s\delta_2}\right]^{-1/s}$

Here the spectral index is still $\alpha$, but we've also introduced _temporal_ power-law indices $\delta_1$ (before the break) and $\delta_2$ (after the break).  $S_{\rm \nu,peak}$ is the flux density at peak, and $s$ controls the smoothness of the break.

Write a function smooth_broken_power_law() that outputs a smoothed broken power law that also scales based on frequency and spectral index

In [ ]:
def smooth_broken_power_law(t, nu, S_peak, t_peak, delta_1, delta_2, alpha, log_s, nu0=3.0):

    
    return 

## Determining evidence for a turnover

We now want to fit a smoothed broken power law to our data and see if there is evidence for a turnover in the radio lightcurve. In our paper we do this via a parameter grid-search to minimise the goodness-of-fit parameter $\chi^2$, i.e., compute $\chi^2(S_{\nu,\rm peak},t_{\rm peak},\delta_1,\delta_2,\alpha,s)$ for a 6-dimension parameter grid. However, grid searches are slow and innefficient.  It's better to concentrate your effort in the part of the fit where the data "prefer" to go.  We can do this using a slightly more complicated statistical technique

Here we will perform an Markov Chain Monte Carlo (MCMC) fit using the [`emcee`](http://dfm.io/emcee/current/) package, to determine lightcurve parameters and the spectral index of the source. First you will need to write 3 functions that define your Probability, Prior and Likelihood.

We will use a uniform prior with $\delta_1>0$ (since we require the lightcurve to initially rise), $0<t_{\rm peak}<300$ (since our data only covers the period up to 200 days) and $s<100$. The parameters will be passed to the function via a tuple.

In [ ]:
def lnprior(theta):
    F_peak, t_peak, delta_1, delta_2, alpha, log_s = theta

    if 0.0 < t_peak < 300.0 and delta_1 > 0.0 and log_s < 3:
        return 0.0
    
    else:
        return -np.inf

We will now write a likelihood function that takes the lightcurve parameters inside the tuple theta, along with the observed data.

In [ ]:
def lnlike(theta, t, nu, S, S_err):
    F_peak, t_peak, delta_1, delta_2, alpha, log_s = theta

    model = smooth_broken_power_law(t, nu, F_peak, t_peak, delta_1, delta_2, alpha, log_s)
    inv_sigma2 = 1.0/S_err**2

    return -0.5*(np.sum((S-model)**2*inv_sigma2 - np.log(inv_sigma2)))

Now write a function to calculate the marginal probability using the lnlike() and lnprior() functions you calculated above

In [ ]:
def lnprob(theta, t, nu, S, S_err):
    lp = lnprior(theta)

    if not np.isfinite(lp):
        return -np.inf

    return lp + lnlike(theta, t, nu, S, S_err)

We will now fit the data using the emcee package. The function get_starting_pos() provided below will set up an array of walker starting positions for given lightcurve parameters. Examine the lightcurve and estimate some reasonable values for these parameters and add them to the function.

Now write a function called run_mcmc() that will load the observed data, take the starting position and then run the emcee Ensemple Sampler. Use a small number of iterations and walkers initially (100/20) to see how long the code takes to run on your laptop. Then increase both parameters to a larger number so that the algorithm takes ~90 seconds to run.

In [ ]:
def get_starting_pos(nwalkers, ndim=6):
    S_peak = 
    t_peak = 
    delta_1 = 
    delta_2 = 
    alpha = 
    log_s = 
    
    pos = [np.asarray([S_peak, t_peak, delta_1, delta_2, alpha, log_s]) + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    
    return pos

In [ ]:
def run_mcmc(data, niters=1000, nthreads=1, nwalkers=200, ndim=6):
    t = data['delta_t']
    nu = data['frequency']
    S = data['flux']
    S_err = data['rms']
    
    pos = get_starting_pos(nwalkers, ndim=ndim)
    
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(t, nu, S, S_err),threads=nthreads)
    
    start = timer()
    sampler.run_mcmc(pos, niters)
    end = timer()
    
    print("Computation time: %f s"%(end-start))
    
    return sampler

We now want to inspect our chain to see if our algorithm has converged to a reasonable solution. First, extract the chain from the sampler, and then write a function to make a figure showing how each walker moves around the parameter space. Your figure should have 6 subplots (1 for each dimension), iteration number on the x-axis and parameter value on the y-axis.

MCMC algorithms typically use a burn-in phase, where the sampler is moving towards the optimum solution and not yet accurately sampling the parameter space. Add a parameter chain_cut to your function that plots a vertical line at the end of the burn-in.

In [ ]:
sampler = run_mcmc(tdata)
chain = sampler.chain

In [ ]:
def make_chain_plot(chain, chain_cut):
    niters = chain.shape[1]
    ndim = chain.shape[2]

    fig, axes = plt.subplots(ndim,1,sharex=True)
    fig.set_size_inches(7, 20)
    
    param_names = ['$S_{{\\nu,\\rm peak}, 3\.{\\rm GHz}}$', '$t_{{\\rm peak}}$','$\\delta_1$','$\\delta_2$', '$\\alpha$', '$\\log_{10}(s)$']

    for i, (ax,param_name) in enumerate(zip(axes,param_names)):
        #plot the chain for the given parameter
        
        
        
        ax.set_ylabel(param_name)
        ax.set_xlim(0,niters)
        
        
        ax.axvline(chain_cut,c='r',linestyle='--')

chain_cut = 

make_chain_plot(chain, chain_cut)

Now that we know that our algorithm is converging, and we know how long the burn-in takes we can begin to estimate parameters. The function below will make a corner plot from the good part of your chain.

In [ ]:
good_chain = chain[:, chain_cut:, :]

In [ ]:
def make_corner_plot(good_chain, savefile='corner.png'):
    param_names = ['$F_{{\\rm peak}, 3\.{\\rm GHz}}$', '$t_{{\\rm peak}}$','$\\delta_1$','$\\delta_2$', '$\\alpha$', '$\\log_{10}(s)$']
    ndim = good_chain.shape[2]
    
    fig = corner.corner(good_chain.reshape((-1, ndim)), labels=param_names, quantiles=[0.16, 0.5, 0.84], show_titles=True)
    plt.savefig(savefile)

make_corner_plot(good_chain)

The function below will then extract the median and uncertainty (1 standard deviation) from the chain.

In [ ]:
def get_best_params(chain):
    ndim = chain.shape[2]
    
    chain = chain.reshape((-1, ndim))
    vals = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(chain, [16, 50, 84],axis=0)))
    
    param_names = ['F_peak', 't_peak', 'delta_1', 'delta_2', 'alpha', 'log_s']
    
    param_dict = dict(zip(param_names,vals))
    
    return param_dict
    
    
best_params = get_best_params(good_chain)

Now write a function, calc_chi2(), that will calculate the $\chi^2$ for the fit. We will use this later to compare different lightcurve models

In [ ]:
def calc_chi2(best_params, param_names, model, data, nu0=3.0):
    args = []
    for param in param_names:
        val = best_params[param][0]
        args.append(val)

    best_fit = model(data['delta_t'], nu0, *args)
    
    chi2 = np.sum((best_fit-data['scaled_flux'])**2/data['scaled_rms']**2)
    
    return chi2

param_names = ['F_peak', 't_peak', 'delta_1', 'delta_2', 'alpha', 'log_s']

chi2_best = calc_chi2(best_params, param_names, smooth_broken_power_law, tdata)
print(chi2_best)

We will now plot our best fit on top of the observational data.

Write a function plot_model() that takes in a function that calculates the model fit (in this case, our smooth_broken_power_law function), the best parameters, an array of values to plot the model for and a matplotlib axis to plot it on. Modify your make_plot function to take in an optional argument for the model to plot. If the model is supplied your make_plot() function should call your plot_model() function.

In [ ]:
def plot_model(model, params, tvals, ax):

    return

plotting_data = scale_data(tdata, best_params['alpha'][0],np.max(best_params['alpha'][1:]))    
    
make_plot(plotting_data,scaled=True,model=smooth_broken_power_law,params=args, plot_models=False)

But how do we know that the lightcurve has definitely turned over?

We can perform a similar process as above to fit a standard power law to our data and then use an **[F-test](https://en.wikipedia.org/wiki/F-test#Regression_problems)** to determine which model (turnover or no turnover) provides the best fit. We have provided a `power_law()` function that calculates the a power-law fit to the data. Now write a series of functions to perform an MCMC fit using a standard power law; `lnprior_noturnover()`, `lnlike_noturnover()`, `lnprob_noturnover()`, `get_starting_pos_noturnover()`, `run_mcmc_noturnover()`.

In [ ]:
def power_law(t, nu, F0, delta_1, alpha, nu0=3.0):
    return (nu/nu0)**alpha * F0 * t**delta_1

def lnprior_noturnover(theta):
    F0, delta_1, alpha = theta

    if delta_1 > 0.0:
        return 0.0
    
    else:
        return -np.inf
    
def lnlike_noturnover(theta, t, nu, S, S_err):
    

    return lnline_val

def lnprob_noturnover(theta, t, nu, S, S_err):
    lp = lnprior_noturnover(theta)

    if not np.isfinite(lp):
        return -np.inf

    return lp + lnlike_noturnover(theta, t, nu, S, S_err)

def get_starting_pos_noturnover(nwalkers, ndim=6):
    F0 = 
    delta_1 = 
    alpha = 
    
    pos = [np.asarray([F0, delta_1, alpha]) + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    
    return pos

def run_mcmc_noturnover(data, niters=1000, nthreads=1, nwalkers=200, ndim=3):
    
    return sampler

Now run the sampler for the standard power law fit and write a function to plot the chains and calculate the length of the burn-in.

In [ ]:
sampler_noturnover = run_mcmc_noturnover(tdata)

In [ ]:
def make_chain_plot_noturnover(chain, chain_cut):
    

chain_noturnover = sampler_noturnover.chain
chain_cut_nt = 

make_chain_plot_noturnover(chain_noturnover, chain_cut)

Make a corner plot for the standard power law fit

In [ ]:
def make_corner_plot_noturnover(good_chain, savefile='corner.png'):
    
    
    plt.savefig(savefile)

good_chain_nt = chain_noturnover[:, chain_cut_nt:, :]

make_corner_plot_noturnover(good_chain_nt)

How do the values of $\delta_1$ and $\alpha$ compare to the previous fit. How does the calculated radio spectral index compare to the spectral index determined from fitting data all the way from the radio to X-rays ($\alpha=-0.585\pm 0.005$; Margutti et al. 2018, ApJ, 856, L18)?

Now write a function to get the best parameters for the standard power law fit.

In [ ]:
def get_best_params_noturnover(chain):
    
    return param_dict
    
    
best_params_nt = get_best_params_noturnover(good_chain_nt)

Now that we have the best parameters for the standard power law fit we can plot it over our data. Don't forget to scale the data based on the calculated best-fit spectral index!

In [ ]:
param_names_nt = ['F0', 'delta_1', 'alpha']
args_nt = []
for param in param_names_nt:
    val = best_params_nt[param][0]
    args_nt.append(val)
        


plotting_data_nt = scale_data(tdata, best_params_nt['alpha'][0],np.max(best_params_nt['alpha'][1:]))    
    
make_plot(plotting_data_nt,scaled=True,model=power_law,params=args_nt)

Calculate the $\chi^2$ for the standard power law fit. We will then use this, and the previously calculated $\chi^2$ to perform an F-test and determine which model we prefer.

In [ ]:
chi2_nt = calc_chi2(best_params_nt, ['F0', 'delta_1', 'alpha'], power_law, tdata)
print(chi2_nt)

An [F-test](https://en.wikipedia.org/wiki/F-test) is a generalised test that can be used to compare statistical models. In particular, it is useful when comparing two models where one is a restricted form of the other. Write a function calculate_ftest that calculates the F statistic for our two fits and then calculates the corresponding p-value. Hint: We have already imported the [scipy.stats F-distribution model](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f.html), and we can access the cumulative distribution function using f.cdf().

In [ ]:
def calculate_ftest(chi2_t, p_t, chi2_nt, p_nt, n):
    
    return 1-pval

n = 
p_t = 
p_nt = 

calculate_ftest(chi2_best, p_t, chi2_nt, p_nt, n)

Which model is preferred? With what confidence can we say that we prefer one model over the other?

## Determining Lightcurve Parameters
We're now going to use the full radio lightcurve to determine the best fitting parameters for the smooth broken power law. Load the full dataset and pass it to your previous run_mcmc() function.

In [ ]:
data = load_data()

full_sampler = run_mcmc(data)
full_chain = full_sampler.chain

Plot the chain and estimate the burn-in length

In [ ]:
full_chain_cut = 200

make_chain_plot(full_chain, full_chain_cut)

Make a corner plot

In [ ]:
good_full_chain = full_chain[:, full_chain_cut:, :]
make_corner_plot(good_full_chain, savefile='corner_fulldata.png')

Calculate the best parameters and the $\chi^2$ for the best fit.

Now plot the full lightcurve. A physically motivated model of a relativistic jet and cocoon of ejecta can also be overplotted by using passing "plot_models=True" to the make_plot function.

Now write a function to plot the best fitting model for the 3 GHz data and call it from your make_plot() function when you pass plot_models=True

In [ ]:
def plot_physical_models(ax, fname='jet_cocoon_contribution.txt'):
    
    return ax

In [ ]:
make_plot(plotting_data, scaled=True, model=smooth_broken_power_law, params=full_args, plot_models=True)

So we have now determined the final temporal/spectral behavior of the radio data.  What does this tell us?  

The fact that the spectral index stayed basically constant over time and that it was the same spectrum from radio to X-ray wavelengths (across ~9 order of magnitude in frequency) tells us that the emission is _optically-thin synchrotron emission_ (Sari et al. 1998, ApJ, 497, L17), which we can use to make other inferences.  We also know that with synchrotron emission you can get a change in the spectrum at high frequencies if the highest-energy electrons have cooled: we see no evidence for a cooling break.  

The temporal behavior can be used to constrain the geometry of the emission.  This is what we (and others) did in a series of papers (**CHECK: are there others to cite?**):
1. [Hallinan et al. (2017)](http://adsabs.harvard.edu/abs/2017Sci...358.1579H)
2. [Alexander et al. (2017)](http://adsabs.harvard.edu/abs/2017ApJ...848L..21A)
3. [Mooley et al. (2018)](http://adsabs.harvard.edu/abs/2018Natur.554..207M)
4. [Margutti et al. (2018)](http://adsabs.harvard.edu/abs/2018ApJ...856L..18M)
5. [Dobie et al. (2018)](http://adsabs.harvard.edu/abs/2018ApJ...858L..15D)
6. [Resmi et al. (2018)](http://adsabs.harvard.edu/abs/2018arXiv180302768R)
7. [Alexander et al. (2018)](http://adsabs.harvard.edu/abs/2018ApJ...863L..18A)

The latest constraints come from Mooley et al. (submitted), which argues that $\alpha$ relates to the intrinsic distribution of relativistic electrons as:
$$\alpha = -\frac{p-1}{2}$$
where the electrons have a power-law distribution of energies with index $p$.  Moreover, we can relate the same $p$ to the temporal index in the decay phase in a way that depends on the geomety:
1. If the emission is dominated by a jet (collimated, relativistic) we expect $\delta_2=-p$ in the decay phase
2. If it is more spherical it should be:
$$\delta_2 = -3\frac{p-1}{4}$$
(Sari et al. 1998 again).

Questions:
1. What do you get for $p$ based on your value of $\alpha$?
2. Which scenario, jet or sphere, do you find best fits in the decay phase?

